# Extracting data from relational databases

## SQLite/MySQL/Microsoft SQL Server Database

### install package

In [13]:
# !conda install -y -q pymysql
!conda install -y -q pymssql

Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/leolin/anaconda3

  added / updated specs: 
    - pymssql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pymssql-2.1.4              |   py36h1de35cc_0         223 KB
    freetds-1.00.91            |       hdbc3d79_0         2.4 MB
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    freetds: 1.00.91-hdbc3d79_0  
    pymssql: 2.1.4-py36h1de35cc_0

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


### prepare

In [ ]:
## start mysql instance locally
# docker run --name mysql -v /Users/leolin/Data/docker/mysql:/var/lib/mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=root  -d mysql:latest
# mysql -e 'create database classroomDB DEFAULT CHARACTER SET utf8 DEFAULT COLLATE utf8_general_ci;'

## start mssql instance locally
# docker run -e 'ACCEPT_EULA=Y' -e 'SA_PASSWORD=P@ssw0rd' -p 1433:1433 --name python-mssql -d microsoft/mssql-server-linux:latest
# create database classroomDB -> go

### import package

In [14]:
# import relationaldb libs
import sqlite3
import pymysql
import pymssql

### connect to database

In [19]:
# create a connection to a database : create the database if not available
## sqlite
# connection = sqlite3.connect("classroomDB.db")

## mysql
# cnx = {
#     'host': '127.0.0.1',
#     'username': 'root',
#     'password': 'root',
#     'db': 'classroomDB'
# }
# connection = pymysql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])

## mssql
cnx = {
    'host': '127.0.0.1:1433',
    'username': 'sa',
    'password': 'P@ssw0rd',
    'db': 'classroomDB'
}
connection = pymssql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
connection.close()

### create table

In [20]:
## open connection
# connection = sqlite3.connect("classroomDB.db")
# connection = pymysql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
connection = pymssql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])

# open cursor
cursor = connection.cursor()

# query for creating table
create_table = """
                CREATE TABLE classroom (
                student_id INTEGER PRIMARY KEY,
                name VARCHAR(20),
                gender CHAR(1),
                physics_marks INTEGER,
                chemistry_marks INTEGER,
                mathematics_marks INTEGER
                );
                """

# execute query
cursor.execute(create_table)

# commit changes
connection.commit()

#close connection
connection.close()


OperationalError: (2714, b"There is already an object named 'classroom' in the database.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n")

### insert data

In [27]:
## sample data
classroom_data = [(1, "Raj", "M", 70, 84, 92),
                  (2, "Poonam", "F", 87, 69, 93),
                  (3, "Nik", "M", 65, 83, 90),
                  (4, "Rahul", "F", 83, 76, 89)]

# connection = sqlite3.connect("classroomDB.db")
# connection = pymysql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
connection = pymssql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
cursor = connection.cursor()

## insert each student record
for student in classroom_data:
## for sqlite/mysql
#     insert_statement = """INSERT INTO classroom(student_id, name, gender, physics_marks, chemistry_marks, mathematics_marks)
#                             VALUES({0}, "{1}", "{2}", {3}, {4}, {5});""".format(student[0], student[1], student[2],
#                                                                           student[3], student[4], student[5])
#     cursor.execute(insert_statement)
## for mssql
    insert_statement = ("""INSERT INTO classroom(student_id, name, gender, physics_marks, chemistry_marks, mathematics_marks)
                            VALUES(%s, %s, %s, %s, %s, %s);""".format(student[0], student[1], student[2],
                                                                          student[3], student[4], student[5]))
    cursor.execute(insert_statement, ((student[0]), (student[1]), (student[2]), (student[3]), (student[4]), (student[5])))

connection.commit()
connection.close()


INSERT INTO classroom(student_id, name, gender, physics_marks, chemistry_marks, mathematics_marks)
                            VALUES(%s, %s, %s, %s, %s, %s);


IntegrityError: (2627, b"Violation of PRIMARY KEY constraint 'PK__classroo__2A33069A5207676A'. Cannot insert duplicate key in object 'dbo.classroom'. The duplicate key value is (1).DB-Lib error message 20018, severity 14:\nGeneral SQL Server error: Check messages from the SQL Server\n")

### extract data

In [26]:
# connection = sqlite3.connect("classroomDB.db")
# connection = pymysql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
connection = pymssql.connect(cnx['host'], cnx['username'], cnx['password'], cnx['db'])
cursor = connection.cursor()
query = "SELECT * FROM classroom"
cursor.execute(query)

#fetch results
result = cursor.fetchall()
#print results
for row in result:
    print(row)

connection.close()

(1, 'Raj', 'M', 70, 84, 92)
(2, 'Poonam', 'F', 87, 69, 93)
(3, 'Nik', 'M', 65, 83, 90)
(4, 'Rahul', 'F', 83, 76, 89)
